### Libraries 

In [2]:
import cv2
import numpy as np

#### Detect

In [8]:
width,height = 400,400
cap = cv2.VideoCapture(1)
cap.set(3,width)
cap.set(4,height)

def empty(a):
    pass

#Create Trackbars to find right parameters for the canny edge detector
cv2.namedWindow("Trackbar")
cv2.resizeWindow("Trackbar",width,200)
cv2.createTrackbar("Canny_Lower","Trackbar",0,255,empty)
cv2.createTrackbar("Canny_Upper","Trackbar",255,255,empty)
cv2.createTrackbar("Area_Lower","Trackbar",0,500000,empty)
cv2.createTrackbar("Area_Upper","Trackbar",500000,500000,empty)

def find_contours(img,imgout):
    
    contours, hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        lower_area = cv2.getTrackbarPos("Area_Lower","Trackbar")
        upper_area = cv2.getTrackbarPos("Area_Upper","Trackbar")
        
        if area>=lower_area and area<=upper_area:
            cv2.drawContours(imgout,cnt,-1,(0,0,255),2)
            perimeter = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt,0.02*perimeter,True)
            no_corners = len(approx)

            
            #Draw bounding rect
            x1,y1,w1,h1 = cv2.boundingRect(approx)
            cv2.rectangle(imgout,(x1,y1),(x1+w1,y1+h1),(255,0,0),5)
            
            cv2.putText(imgout,"Vertices: "+str(no_corners),(x1+int(w1/2)+30,y1+int(h1/2)+30),cv2.FONT_HERSHEY_COMPLEX,
                        0.8,(25,200,100),2)
            
            cv2.putText(imgout,"Area: "+str(area),(50,50),cv2.FONT_HERSHEY_COMPLEX,
                       0.8,(0,0,255),2)
            
        else:
            continue
            



while True:
    success,img = cap.read()
    imgout = img.copy()
    img_blur = cv2.GaussianBlur(img,(3,3),1)
    img_gray = cv2.cvtColor(img_blur,cv2.COLOR_BGR2GRAY)
    
    re_ht,re_wd = 300,300
    

    
    
    lower = cv2.getTrackbarPos("Canny_Lower","Trackbar")
    upper = cv2.getTrackbarPos("Canny_Upper","Trackbar")
    
    #Applying Canny Edge Detector
    canny = cv2.Canny(img_gray,lower,upper)
    
    #Apply Dilation to remove some noise
    dilate = cv2.dilate(canny,np.ones(shape = (7,7),dtype=float),iterations = 1)
    
    
    #Detecting contours
    find_contours(dilate,imgout)
    
     
    #Display Image
    
    stacked = np.hstack([cv2.resize(img,(re_ht,re_wd)),
                         cv2.resize(imgout,(re_ht,re_wd)),
                         cv2.cvtColor(cv2.resize(dilate,(re_ht,re_wd)),cv2.COLOR_GRAY2BGR)])
    
    
    
    cv2.imshow("Stacked",stacked)
    
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()